In [5]:
! pip install flask nltk
from flask import Flask, request, render_template_string
import sqlite3
from nltk.chat.util import Chat, reflections
from werkzeug.serving import run_simple


In [6]:
conn = sqlite3.connect('classroom_discussion.db')
conn.execute('CREATE TABLE IF NOT EXISTS discussions (id INTEGER PRIMARY KEY, user TEXT, question TEXT, response TEXT)')
conn.close()


In [7]:
pairs = [
  (r'(.*)(assignment|homework)(.*)', ['Assignments are due on Fridays.']),
  (r'(.*)(lecture|topic)(.*)', ['Today’s lecture covered Machine Learning basics.']),
  (r'hello|hi|hey', ['Hello! How can I assist you?']),
  (r'(.*)', ['I’m sorry, I don’t have an answer for that.'])
]

chatbot = Chat(pairs, reflections)


In [8]:
app = Flask(__name__)


In [9]:
@app.route('/')
def home():
  html_content = """
  <!DOCTYPE html>
  <html>
  <head><title>Classroom Bot</title></head>
  <body>
    <h1>Classroom Discussion Bot</h1>
    <form action="/get_response" method="post">
      <input type="text" name="message" placeholder="Ask a question" required>
      <button type="submit">Submit</button>
    </form>
    <div id="chat-history"></div>
  </body>
  </html>
  """
  return render_template_string(html_content)


In [10]:
@app.route('/get_response', methods=['POST'])
def get_response():
  user_input = request.form['message']  # Get user question from form
  response = chatbot.respond(user_input)  # Get bot's response
  
  # Save the interaction in the SQLite database
  conn = sqlite3.connect('classroom_discussion.db')
  conn.execute('INSERT INTO discussions (user, question, response) VALUES (?, ?, ?)', ('Student', user_input, response))
  conn.commit()
  conn.close()
  
  # Return a simple HTML showing the user question and bot response
  return f"your question: {user_input}<br>chat: {response}"


In [11]:
if __name__ == '__main__':
  run_simple('localhost', 5000, app, use_reloader=False)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Oct/2025 06:48:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 06:48:47] "POST /get_response HTTP/1.1" 200 -
